<a href="https://colab.research.google.com/github/alaa1996001/Hello_world/blob/master/multithreading_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Threads Harmony

Lets say that we have 2 building compagnies: A and B. They all have separate work schedules composed with simple tasks, that the Company can achieve by itself, or a complex task, that can only be achieved by getting help from another Company. Each task have a random duration between 1 and 5 seconds.
If a Company works on more than one task, it doubles the duration of all the current tasks it's working on.

In [ ]:
import threading
import random 
import time

class Task(object):
    def __init__(self,name,help_=None):
        self.name = name
        self.help = help_
        self.doubled = False
        self.duration = random.randint(1,5)
        
    def double_duration(self):
        if not self.doubled:
            self.duration = 2*self.duration; self.doubled = True
            print(self.name+" duration doubled")
            
    def get_duration(self):
        if self.doubled:
            return self.duration/2
        return self.duration
            
    def start(self):
        self.time_start = time.time()
        
    def ended(self):
        return time.time() > self.time_start+self.duration

class Company(threading.Thread):
    def __init__(self,name):
        super(Company,self).__init__(daemon=True)
        self.name = name
        self.working_on = []
        
    def set_schedule(self,schedule):
        self.schedule = schedule
        
    def work_on(self,task):
        self.working_on.append(task)
        task.start()
        print(self.name+" working on "+task.name+" (duration {}s)".format(task.duration))
        if len(self.working_on) > 1:
            [task.double_duration() for task in self.working_on]
        
    def complete_task(self,task):
        print(self.name+" has completed "+task.name)
        self.working_on.remove(task)

###### Without a lock 

In [ ]:
class Company_without_a_lock(C):
    def run(self):
        start_time= time.time()
        for task in self.schedule:
            if task.help is not None:
                task.help.work_on(task)
            self.work_on(task)
            while len(self.working_on) > 0:
                for tsk in self.working_on:
                    if tsk.ended():
                        self.complete_task(tsk)
                time.sleep(0.1)
        self.duration = time.time()-start_time 

SCENARIO 1A: No help is needed for any task

In [ ]:
Company_A_TASKS = [
    Task("Project A"),
    Task("Project B"),
    Task("Project C"),
]
Company_B_TASKS = [
    Task("Project D"),
    Task("Project E"),
    Task("Project F"),
]

Company_A = Company_without_a_lock("Company A");Company_A.set_schedule(Company_A_TASKS);
Company_B = Company_without_a_lock("Company B");Company_B.set_schedule(Company_B_TASKS);

Company_A.start();
Company_B.start();

Company_A.join()
Company_B.join()

print('Company A minimum time possible: ',sum([task.get_duration() for task in Company_A.schedule])+sum([task.get_duration() for task in Company_B.schedule if task.help is Company_A]))
print('Company A actual working time: ',Company_A.duration)
print('Company B minimum time possible: ',sum([task.get_duration() for task in Company_B.schedule])+sum([task.get_duration() for task in Company_A.schedule if task.help is Company_B]))
print('Company B actual working time: ',Company_B.duration)

Compagny A working on Project A (duration 3s)Compagny B working on Project D (duration 2s)

Compagny B has completed Project D
Compagny B working on Project E (duration 1s)
Compagny A has completed Project A
Compagny B has completed Project ECompagny A working on Project B (duration 1s)

Compagny B working on Project F (duration 1s)
Compagny A has completed Project B
Compagny B has completed Project FCompagny A working on Project C (duration 1s)

Compagny A has completed Project C
Compagny A minimum time possible:  5
Compagny A actual working time:  5.435757160186768
Compagny B minimum time possible:  4
Compagny B actual working time:  4.407822132110596


SCENARIO 1B: Simple Schedule with some tasks that needs collaboration

In [ ]:
Company_A = Company_without_a_lock("Company A");
Company_B = Company_without_a_lock("Company B");

Company_A_TASKS = [
    Task("Project A"),
    Task("Project B",help_=Company_B),
    Task("Project C"),
]
Company_B_TASKS = [
    Task("Project D"),
    Task("Project E"),
    Task("Project F",help_=Company_A),
]

Company_A.set_schedule(Company_A_TASKS);
Company_B.set_schedule(Company_B_TASKS);

Company_A.start();
Company_B.start();

Company_A.join()
Company_B.join()

print('Company A minimum time possible: ',sum([task.get_duration() for task in Company_A.schedule])+sum([task.get_duration() for task in Company_B.schedule if task.help is Company_A]))
print('Company A actual working time: ',Company_A.duration)
print('Company B minimum time possible: ',sum([task.get_duration() for task in Company_B.schedule])+sum([task.get_duration() for task in Company_A.schedule if task.help is Company_B]))
print('Company B actual working time: ',Company_B.duration)

Compagny A working on Project A (duration 3s)Compagny B working on Project D (duration 2s)

Compagny B has completed Project D
Compagny B working on Project E (duration 5s)
Compagny A has completed Project A
Compagny B working on Project B (duration 4s)
Project E duration doubled
Project B duration doubled
Compagny A working on Project B (duration 8s)
Compagny A has completed Project BCompagny B has completed Project B

Compagny A working on Project C (duration 1s)
Compagny B has completed Project E
Compagny A has completed Project CCompagny A working on Project F (duration 1s)

Compagny B working on Project F (duration 1s)
Compagny B has completed Project FCompagny A has completed Project F

Compagny A minimum time possible:  9.0
Compagny A actual working time:  13.499382019042969
Compagny B minimum time possible:  12.0
Compagny B actual working time:  13.499294996261597


###### With a lock 

In [ ]:
class Company_with_a_lock(Company):
    def __init__(self,name,lock):
        super(Company_with_a_lock,self).__init__(name)
        self.lock = lock
        
    def run(self):
        self.time_waiting = 0
        start_time= time.time()
        copy = self.schedule[:]
        while len(copy) > 0 or len(self.working_on) > 0:
            while len(self.working_on) > 0:
                for task in self.working_on:
                    if task.ended():
                        self.complete_task(task)
                time.sleep(0.1)
            try:
                self.lock.release()
            except:
                pass
            time.sleep(0.5) #Wait 0.5 to see if another Company needs help
            self.lock.acquire()
            if len(self.working_on) == 0 and len(copy) > 0:
                task = copy.pop(0)
                if task.help is not None:
                    tm = time.time()
                    task.help.lock.acquire()
                    self.time_waiting += (time.time()-tm)
                    task.help.work_on(task)
                    task.help.lock.release()
                self.work_on(task)
        try:
            self.lock.release()
        except:
            pass
        self.duration = time.time()-start_time 

SCENARIO 2A: No help is needed for any task

In [ ]:
Company_A_TASKS = [
    Task("Project A"),
    Task("Project B"),
    Task("Project C"),
]
Company_B_TASKS = [
    Task("Project D"),
    Task("Project E"),
    Task("Project F"),
]

Company_A = Company_with_a_lock("Company A",threading.Lock());Company_A.set_schedule(Company_A_TASKS);
Company_B = Company_with_a_lock("Company B",threading.Lock());Company_B.set_schedule(Company_B_TASKS);

Company_A.start();
Company_B.start();

Company_A.join()
Company_B.join()

print('Company A minimum time possible: ',sum([task.get_duration() for task in Company_A.schedule])+sum([task.get_duration() for task in Company_B.schedule if task.help is Company_A]))
print('Company A working time: ',Company_A.duration-Company_A.time_waiting)
print('Company A waiting time: ',Company_A.time_waiting)
print('Company A total time: ',Company_A.duration)
print('Company B minimum time possible: ',sum([task.get_duration() for task in Company_B.schedule])+sum([task.get_duration() for task in Company_A.schedule if task.help is Company_B]))
print('Company B working time: ',Company_B.duration-Company_B.time_waiting)
print('Company B waiting time: ',Company_B.time_waiting)
print('Company B total time: ',Company_B.duration)

Compagny A working on Project A (duration 5s)
Compagny B working on Project D (duration 4s)
Compagny B has completed Project D
Compagny B working on Project E (duration 1s)
Compagny A has completed Project A
Compagny B has completed Project E
Compagny A working on Project B (duration 3s)
Compagny B working on Project F (duration 1s)
Compagny B has completed Project F
Compagny A has completed Project B
Compagny A working on Project C (duration 2s)
Compagny A has completed Project C
Compagny A minimum time possible:  10
Compagny A working time:  12.539860963821411
Compagny A waiting time:  0
Compagny A total time:  12.539860963821411
Compagny B minimum time possible:  6
Compagny B working time:  8.39522409439087
Compagny B waiting time:  0
Compagny B total time:  8.39522409439087


SCENARIO 2B: Simple Schedule with some tasks that needs collaboration

In [ ]:
Company_A = Company_with_a_lock("Company A",threading.Lock());
Company_B = Company_with_a_lock("Company B",threading.Lock());

Company_A_TASKS = [
    Task("Project A"),
    Task("Project B",help_=Company_B),
    Task("Project C"),
]
Company_B_TASKS = [
    Task("Project D"),
    Task("Project E"),
    Task("Project F",help_=Company_A),
]

Company_A.set_schedule(Company_A_TASKS);
Company_B.set_schedule(Company_B_TASKS);

Company_A.start();
Company_B.start();

Company_A.join()
Company_B.join()

print('Company A minimum time possible: ',sum([task.get_duration() for task in Company_A.schedule])+sum([task.get_duration() for task in Company_B.schedule if task.help is Company_A]))
print('Company A working time: ',Company_A.duration-Company_A.time_waiting)
print('Company A waiting time: ',Company_A.time_waiting)
print('Company A total time: ',Company_A.duration)
print('Company B minimum time possible: ',sum([task.get_duration() for task in Company_B.schedule])+sum([task.get_duration() for task in Company_A.schedule if task.help is Company_B]))
print('Company B working time: ',Company_B.duration-Company_B.time_waiting)
print('Company B waiting time: ',Company_B.time_waiting)
print('Company B total time: ',Company_B.duration)

Compagny A working on Project A (duration 4s)
Compagny B working on Project D (duration 1s)
Compagny B has completed Project D
Compagny B working on Project E (duration 1s)
Compagny B has completed Project E
Compagny A has completed Project A
Compagny A working on Project F (duration 4s)
Compagny B working on Project F (duration 4s)
Compagny B has completed Project FCompagny A has completed Project F

Compagny B working on Project B (duration 4s)
Compagny A working on Project B (duration 4s)
Compagny B has completed Project BCompagny A has completed Project B



### Computing time loss per scenario

#### Scenario 1B

In [ ]:
import numpy  as np

time_losses = []
time_losses_perc = []
for i in range(0,10):
    print('iteration',i,'\n\n')
    Company_A_TASKS = [
        Task("Project A"),
        Task("Project B"),
        Task("Project C"),
    ]
    Company_B_TASKS = [
        Task("Project D"),
        Task("Project E"),
        Task("Project F"),
    ]

    Company_A = Company_without_a_lock("Company A");Company_A.set_schedule(Company_A_TASKS);
    Company_B = Company_without_a_lock("Company B");Company_B.set_schedule(Company_B_TASKS);

    Company_A.start();
    Company_B.start();

    Company_A.join()
    Company_B.join()
    
    min_a = sum([task.get_duration() for task in Company_A.schedule])+sum([task.get_duration() for task in Company_B.schedule if task.help is Company_A])
    min_b = sum([task.get_duration() for task in Company_B.schedule])+sum([task.get_duration() for task in Company_A.schedule if task.help is Company_B])
    time_losses.append(Company_A.duration-min_a)
    time_losses.append(Company_B.duration-min_b)
    time_losses_perc.append(100*(Company_A.duration/min_a))
    time_losses_perc.append(100*(Company_B.duration/min_b))
    
mean_loss = np.mean(time_losses)
mean_loss_p = np.mean(time_losses_perc)
print("Mean time lost: ",mean_loss,"s")
print("Mean time spent: ",str(mean_loss_p)[:5],"% of minimum time")

#### Scenario 1B

In [ ]:
import numpy  as np

time_losses = []
time_losses_perc = []
for i in range(0,10):
    print('iteration',i,'\n\n')
    Company_A = Company_without_a_lock("Company A");
    Company_B = Company_without_a_lock("Company B");

    Company_A_TASKS = [
        Task("Project A"),
        Task("Project B",help_=Company_B),
        Task("Project C"),
    ]
    Company_B_TASKS = [
        Task("Project D"),
        Task("Project E"),
        Task("Project F",help_=Company_A),
    ]

    Company_A.set_schedule(Company_A_TASKS);
    Company_B.set_schedule(Company_B_TASKS);

    Company_A.start();
    Company_B.start();

    Company_A.join()
    Company_B.join()
    
mean_loss = np.mean(time_losses)
mean_loss_p = np.mean(time_losses_perc)
print("Mean time lost: ",mean_loss,"s")
print("Mean time spent: ",str(mean_loss_p)[:5],"% of minimum time")

#### Scenario 2A

In [ ]:
import numpy  as np

time_losses = []
time_losses_perc = []
for i in range(0,10):
    print('iteration',i,'\n\n')
    Company_A = Company_with_a_lock("Company A",threading.Lock());
    Company_B = Company_with_a_lock("Company B",threading.Lock());

    Company_A_TASKS = [
        Task("Project A"),
        Task("Project B"),
        Task("Project C"),
    ]
    Company_B_TASKS = [
        Task("Project D"),
        Task("Project E"),
        Task("Project F"),
    ]

    Company_A.set_schedule(Company_A_TASKS);
    Company_B.set_schedule(Company_B_TASKS);

    Company_A.start();
    Company_B.start();

    Company_A.join()
    Company_B.join()
    
mean_loss = np.mean(time_losses)
mean_loss_p = np.mean(time_losses_perc)
print("Mean time lost: ",mean_loss,"s")
print("Mean time spent: ",str(mean_loss_p)[:5],"% of minimum time")

#### Scenario 1B

In [ ]:
import numpy  as np

time_losses = []
time_losses_perc = []
for i in range(0,10):
    print('iteration',i,'\n\n')
    Company_A = Company_with_a_lock("Company A",threading.Lock());
    Company_B = Company_with_a_lock("Company B",threading.Lock());

    Company_A_TASKS = [
        Task("Project A"),
        Task("Project B",help_=Company_B),
        Task("Project C"),
    ]
    Company_B_TASKS = [
        Task("Project D"),
        Task("Project E"),
        Task("Project F",help_=Company_A),
    ]

    Company_A.set_schedule(Company_A_TASKS);
    Company_B.set_schedule(Company_B_TASKS);

    Company_A.start();
    Company_B.start();

    Company_A.join()
    Company_B.join()
    
mean_loss = np.mean(time_losses)
mean_loss_p = np.mean(time_losses_perc)
print("Mean time lost: ",mean_loss,"s")
print("Mean time spent: ",str(mean_loss_p)[:5],"% of minimum time")

## Excercice

Figure out how to achieve synchronisation between compagnies using semaphores